In [1]:
import chess
import chess.svg
import random
from IPython.display import display, SVG
import json
from time import sleep
import os
import subprocess

In [2]:
def display_board(board, size=500):
    svg_board = chess.svg.board(board, size=size)
    display(SVG(svg_board))

def random_move_game(board, num_moves=20):
    for _ in range(num_moves):
        legal_moves = list(board.legal_moves)
        if not legal_moves:
            break
        move = random.choice(legal_moves)
        board.push(move)
    return board

def numPieces( bitMap ):

   def sum_bits(num):
      return bin(num).count('1')

   return sum_bits(bitMap["rooks"]) + \
            sum_bits(bitMap["knights"]) + \
            sum_bits(bitMap["bishops"]) + \
            sum_bits(bitMap["queens"]) + \
            sum_bits(bitMap["kings"]) + \
            sum_bits(bitMap["pawns"])

def getPosFromNotation(notation: str, isWhite:bool):
    horizontalPos = ord(notation[1]) - ord('1')
    verticalPos = ord(notation[0]) - ord('a')
    return 1 << ((64*(not isWhite)) + (63 - (horizontalPos*8 + verticalPos)))


def isSameColorPiecePresent(board, pos, isWhiteMove:bool):
   for pieces in board:
      if board[pieces] & (1<<(64*(not isWhiteMove) + pos)):
         return True
   return False


def isOppositeColorPiecePresent(board, pos, isWhiteMove:bool):
    return isSameColorPiecePresent(board, pos, not isWhiteMove)

def isMovePossible(board, pos, isWhiteMove: bool):
   return not (pos >=64 or pos < 0 or isSameColorPiecePresent(board, pos, isWhiteMove))

def boardToBitMap( board ):

   pieceStructMap = {
      "r": "rooks",
      "n": "knights",
      "b": "bishops",
      "k": "kings",
      "q": "queens",
      "p": "pawns",
   }

   boardBitMap = {
      "rooks": 0,
      "knights": 0,
      "bishops": 0,
      "queens": 0,
      "kings": 0,
      "pawns": 0,
      "metadata": 0,
   }

   for index, piece in board.piece_map().items():
      isBlack = piece.symbol().islower()
      effIndex = 64*isBlack + ( 63 - index )
      boardBitMap[ pieceStructMap[ piece.symbol().lower() ] ] |= 1 << effIndex

   metadata = 0
   metadata |= board.fullmove_number << 16
   metadata |= board.halfmove_clock << 9
   metadata |= board.turn << 8
   en_passant_square = board.ep_square
   metadata |= bool( en_passant_square ) << 7
   if en_passant_square:
      column_number = chess.square_file(en_passant_square)
      metadata |= column_number << 4
   metadata |= board.has_kingside_castling_rights(0) << 3
   metadata |= board.has_queenside_castling_rights(0) << 2
   metadata |= board.has_kingside_castling_rights(1) << 1
   metadata |= board.has_queenside_castling_rights(1) << 0
   boardBitMap[ 'metadata' ] = metadata

   return boardBitMap

def bitMapFile( fileName, bitMap=None, isRead=True ):
   if isRead:
      with open(fileName, "r+") as f: 
         bitMap = json.load(f)
         return bitMap
   else:
      assert bitMap is not None, "No bitMap provided  to save!"
      with open(fileName, "w+") as f: 
         json.dump(bitMap, f, indent=2)

def bitMapToBoard( bitMap ):
   chessPieceMap = {
      "rooks": chess.ROOK,
      "knights": chess.KNIGHT,
      "bishops": chess.BISHOP,
      "kings": chess.KING,
      "queens": chess.QUEEN,
      "pawns": chess.PAWN
   }

   chessBoard = chess.Board()
   chessBoard.clear_board()

   for key, value in bitMap.items():
      if key != 'metadata':
         for index in range(127, -1, -1):
            isPresent = value & 1 << index
            if isPresent:
               isWhite = index <= 63
               effIndex = 63 - ( index - ( not isWhite ) * 64 )
               chessPiece = chess.Piece( chessPieceMap[ key ], isWhite )
               chessBoard.set_piece_at( effIndex, chessPiece )
   
   castlingFen, metadata = '', bitMap[ 'metadata' ]
   chessBoard.fullmove_number = metadata >> 16
   chessBoard.halfmove_clock = metadata >> 9 & 127
   chessBoard.turn = bool( metadata & 1 << 8 )
   enPassantSquareExists = metadata & 1 << 7
   if enPassantSquareExists:
      rowNum = 5 if chessBoard.turn else 2
      chessBoard.ep_square = chess.square( metadata >> 4 & 7, rowNum )
   if metadata & 1 << 1: castlingFen += 'K'
   if metadata & 1 << 0: castlingFen += 'Q'
   if metadata & 1 << 3: castlingFen += 'k'
   if metadata & 1 << 2: castlingFen += 'q'
   chessBoard.set_castling_fen( castlingFen )

   return chessBoard

def display_board(board, size=500):
    svg_board = chess.svg.board(board, size=size)
    display(SVG(svg_board))

In [13]:
chessBoard = chess.Board()

invalidCount = 7
while invalidCount and not chessBoard.is_checkmate():

   display_board( chessBoard )
   while invalidCount:
      try: 
         user_move = input( "User Move:" )
         chessBoard.push_san( user_move )
         break
      except:
         print("Invalid Move!!", user_move )
         invalidCount -= 1
   bitMapFile( "./sample/position.json", boardToBitMap( chessBoard ), isRead=False )
   clear_output()
   display_board( chessBoard )

   sleep( 1 )
   result = subprocess.run(
        ["./target/release/mystic-bot"], 
        check=True, 
        capture_output=True, 
        text=True
    )
   chessBoard = bitMapToBoard( bitMapFile( "./sample/position.json" ) )
   clear_output()
   print("Output:\n", result.stdout)

result = chessBoard.result()

if result == "1-0":
    print("White wins!")
elif result == "0-1":
    print("Black wins!")
elif result == "1/2-1/2":
    print("The game is a draw!")
else:
    print("The game is still ongoing.")

Output:
 Evaluation Function: 0
Board saved successfully to ./sample/position.json

The game is still ongoing.
